In [1]:
%matplotlib inline

In [2]:
import scipy as sp
import scipy.interpolate
import numpy as np
import glob #to read the files
from sklearn.decomposition import PCA
from ipywidgets import *
from tabulate import tabulate #to export in table format
from scipy import interpolate
import scipy.interpolate as interp
from scipy.integrate import quad
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
import scipy.integrate as integrate
from astropy import constants as const
import sncosmo
import json
from astropy.time import Time #to time converting
from astropy import units #time correction
from astropy.coordinates import SkyCoord #time correction
import math

In [3]:
%run /home/joao/Documentos/templates/functions.ipynb

In [4]:
csp = sncosmo.get_magsystem('csp')
Norm_V = (1/(csp.zpbandflux("cspv9844")))
Norm_r = (1/(csp.zpbandflux("cspr")))

In [5]:
Norm_V

1.6839572518342795e-06

In [6]:
filter_y_V = []
filter_z_V = []

In [7]:
file = open("V_tel_ccd_atm_ext_1.2.txt")
lines = file.readlines()
file.close()

In [8]:
for line in lines:

    if line.startswith('#'): continue
    co=line.rstrip().replace('INDEF','Nan').split()

    filter_y_V.append(co[0])
    filter_z_V.append(co[1])

filter_y_V = np.array(filter_y_V, dtype=float)
filter_z_V = np.array(filter_z_V, dtype=float)

In [9]:
f_filter_V = interp1d(filter_y_V, filter_z_V)

In [10]:
filter_y_r = []
filter_z_r = []

In [11]:
file = open("r_tel_ccd_atm_ext_1.2_new.txt")
lines = file.readlines()
file.close()

In [12]:
for line in lines:

    if line.startswith('#'): continue
    co=line.rstrip().replace('INDEF','Nan').split()

    filter_y_r.append(co[0])
    filter_z_r.append(co[1])

filter_y_r = np.array(filter_y_r, dtype=float)
filter_z_r = np.array(filter_z_r, dtype=float)

In [13]:
f_filter_r = interp1d(filter_y_r, filter_z_r)

In [14]:
x = np.linspace(-10,50,61)
y = np.linspace(3000,9000,601)

In [15]:
file = open("max_values.txt")
lines = file.readlines()
file.close()

snname = []
maxvalueV = []
maxvaluer = []

for line in lines:

    if line.startswith('#'): continue
    co=line.rstrip().replace('INDEF','Nan').split()

    snname.append(co[0])
    maxvalueV.append(co[1])
    maxvaluer.append(co[2])


maxvalueV = np.array(maxvalueV, dtype=float)
maxvaluer = np.array(maxvaluer, dtype=float)

In [16]:
#for i in range(0,len(snname)):
    
#    snname[i] = snname[i][:-5]

In [17]:
sn_names=glob.glob("*.dat")
print(len(sn_names))

134


In [18]:
for i in range(0,len(sn_names)):

    sn_names[i] = sn_names[i][:-4]

In [19]:
factorsV = []
namesV = []

In [20]:
factorsr = []
namesr = []

In [21]:
#hist = []

In [22]:
#sn_names = ["SN2012fr"]

In [23]:
for k in range(0,len(sn_names)):
    
    name = "/home/joao/Documentos/templates/"+sn_names[k]+".json"

    #print(name)
    
    with open(name, "r") as read_file:
        data = json.load(read_file)

    
    redshift = redshift_value(data[sn_names[k]])
    
    
    '''
    filter_y_V_temp = []
    
    filter_y_r_temp = []
    
    for i in range(0, len(filter_y_V)):
        
        filter_y_V_temp.append(filter_y_V[i]*(1/(1+redshift)))
        
    for i in range(0, len(filter_y_r)):
        
        filter_y_r_temp.append(filter_y_r[i]*(1/(1+redshift))) 
    
    
    f_filter_V = interp1d(filter_y_V_temp, filter_z_V)
    f_filter_r = interp1d(filter_y_r_temp, filter_z_r)
    '''
    x_lc = []
    z_lc = []

    #print(sn_names[k])

    N = sn_names[k]+".dat"


    file = open(N)
    lines = file.readlines()
    file.close()

    xtemp = []

    for line in lines:

        if line.startswith('#'): continue
        co=line.rstrip().replace('INDEF','Nan').split()

        xtemp.append(co[0])

    xtemp = np.array(xtemp, dtype=float)


    z = np.reshape(xtemp, (601, 61))

    zz = []
    xx = []
    yy = []

    for i in range(0,len(y)):

        for j in range(0,len(x)):

            zz.append(z[i][j])
            xx.append(x[j])
            yy.append(y[i])
            
            
    #print(zz)        

    wavetest = []
    
    for i in range(0,len(zz)):
    
        if zz[i] != 0:

            wavetest.append(yy[i])
    
    minvar = False
    maxvar = False
    
    #teste da banda V
    for i in range(0,len(wavetest)):

        if 4600*(1/(1+redshift)) - 20 < wavetest[i] < 4600*(1/(1+redshift)) + 20:
            minvar = True
            #print("possui minimo")

        if 6580*(1/(1+redshift)) - 20 < wavetest[i] < 6580*(1/(1+redshift)) + 20:
            maxvar = True
            #print("possui maximo")
    #print(minvar, maxvar)
    
    
    if maxvar == True and minvar == True:
        
        for j in range(0,len(x)):

            y_temp = []
            z_temp = []

            for i in range(0,len(xx)):

                if xx[i] == x[j]:

                    y_temp.append(yy[i])
                    z_temp.append(zz[i])
            
            f = interp1d(y_temp, z_temp)

            def S(x): return (1+redshift)*f(x)*f_filter_V(x)*x
            
            integral_value = integrate.quad(S, min(filter_y_V), max(filter_y_V))[0]
            
            z_lc.append(integral_value*Norm_V)
            
            x_lc.append(x[j]) 

        maxvaluecalc = max(z_lc)

        #print(maxvaluecalc)
        if sn_names[k] in snname:
            
            idx = snname.index(sn_names[k])
            
            if maxvalueV[idx] != -9999:

                #print("ok")
                factorsV.append(maxvalueV[idx]/maxvaluecalc)
                namesV.append(sn_names[k])
            
    # se nao ta no range da banda v entao tenta r
    if maxvar == False or minvar == False:
        
        print("nao ta na banda v")
        minvar = False
        maxvar = False

        #teste da banda V
        for i in range(0,len(wavetest)):

            if 5340*(1/(1+redshift)) - 20 < wavetest[i] < 5340*(1/(1+redshift)) + 20:
                minvar = True
                #print("possui minimo")

            if 7190*(1/(1+redshift)) - 20 < wavetest[i] < 7190*(1/(1+redshift)) + 20:
                maxvar = True
                #print("possui maximo")
        #print(minvar, maxvar)
        
        if maxvar == True and minvar == True:
                print("teste para r sucesso")
                for j in range(0,len(x)):

                    y_temp = []
                    z_temp = []

                    for i in range(0,len(xx)):

                        if xx[i] == x[j]:

                            y_temp.append(yy[i])
                            z_temp.append(zz[i])

                    f = interp1d(y_temp, z_temp)

                    def S(x): return (1+redshift)*f(x)*f_filter_r(x)

                    integral_value = integrate.quad(S, min(filter_y_r), max(filter_y_r))[0]

                    z_lc.append(integral_value*Norm_r)

                    x_lc.append(x[j])

                maxvaluecalc = max(z_lc)

                if sn_names[k] in snname:

                    idx = snname.index(sn_names[k])

                    if maxvaluer[idx] != -9999:

                        #print("ok")
                        factorsr.append(maxvaluer[idx]/maxvaluecalc)
                        namesr.append(sn_names[k])

0.02731
0.007929
0.03024
0.037039
0.0109
0.006004
0.00885
nao ta na banda v
0.015067397898713919
0.034901
0.025007
0.000841
0.006861
0.008529
0.01433
0.01043
0.01558
0.02406
0.017432
0.005101
0.007132
0.020071
0.02197002237988821
0.006941
0.02660272499077032
0.007609
0.02834
0.023591
0.003889
0.0115
0.026147
0.01577
0.00948
0.008309
0.01914
0.02077
0.00488
0.01117
0.01306
0.00749
0.019207
0.016455
0.006461
0.032156
0.01490009610926979
0.00897
0.01388
0.023131
0.02349
0.01973
0.032102
0.01354
0.03096718391784581
0.001358
0.008569
0.037149
0.015871
0.01744
0.00524
0.004266
0.001458
0.01660210068034762
0.007899
0.00424
0.01647
0.006494
0.0259


<ipython-input-23-cedc70d99f6f>:114: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  integral_value = integrate.quad(S, min(filter_y_V), max(filter_y_V))[0]


0.045948829040162176
0.00626
0.031
0.03
0.028116
0.004483
0.015501
0.007122
0.004069
0.00149
0.0239
0.014046
0.010344
0.015854
0.02629016098230008
0.009393
0.00864
0.002128
0.006211
0.01444
0.003699
0.016725
0.02829
0.01929
0.011639
0.031003287895856158
0.007722
0.008
0.009316
0.006561
0.006721
0.012526589416171241
0.031021
0.005656012429203772
0.015567
0.016317
0.003662
0.000804
0.02099
0.0054
0.005464
0.022289
0.041045
0.02197499768617539
0.01411
0.033283
0.01158
0.058
0.01512
0.01502
0.012946957105361312
0.034601
0.022402
0.07423915994326902
0.007388
0.01141
0.068
0.006381
0.003559
nao ta na banda v
0.017653315868428443
0.02158
0.006174
0.013012
0.013539
0.004623
nao ta na banda v
teste para r sucesso
0.02129693315973835
0.013189
0.024027


In [24]:
namesr

['SN2008ha']

In [25]:
save_path = '/home/joao/Documentos/PCA'

In [26]:

table = []
for l in range(0,len(namesV)):
    table.append((namesV[l], factorsV[l]))

name_of_file = "factors"

completeName = os.path.join(save_path, name_of_file+".txt")

f = open(completeName, 'w')
f.write(tabulate(table, tablefmt="plain"))
f.close()
